In [48]:
sc

In [49]:
import json 
import re
import os
import glob

In [50]:
def collection(df):
    subjectCollection = open('SubjectCollection.txt', 'a')
    personsCollection = open('PersonsCollection.txt', 'a')
    organizationCollection = open('OrganizationCollection.txt', 'a')
    #creativeCollection = open('CreativeCollection.txt', 'a')
    
    for i in range(len(df)):
        for j in range(len(df[i])):
            if df[i][j][1] == 'subject':
                subject = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                for word in subject.split():
                    subjectCollection.write(word+'\n')
                
            elif df[i][j][1] == 'persons':
                person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                personsCollection.write(person+'\n')
            
            elif df[i][j][1] == 'organizations':
                person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                organizationCollection.write(person+'\n')
# DONT NEED            
#             elif df[i][j][1] == 'creative_works':
#                 person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
#                 creativeCollection.write(person+'\n')
            
#             else:
#                 print df[i][j][1]
                
    subjectCollection.close()
    personsCollection.close()
    organizationCollection.close()

In [51]:
#archive = open('archivejan1.json', 'r')
# os.listdir('~')
# df.show()
# df.printSchema()
cwd = os.getcwd()
print cwd
for i in  glob.glob(cwd+'/archive*'):
    df = spark.read.json(i)
    df = df.select('response.docs.keywords')
    df = df.first()[0]
    collection(df)

/Users/Enzo/anaconda2/CSC_599_WORK


In [41]:
print df[0]
print "======================================"
print df[0][0]
print len(df)
print len(df[0])

[Row(is_major=u'N', name=u'subject', rank=u'1', value=u'Cyberwarfare and Defense'), Row(is_major=u'N', name=u'subject', rank=u'2', value=u'Presidential Election of 2016'), Row(is_major=u'N', name=u'persons', rank=u'3', value=u'Merkel, Angela'), Row(is_major=u'N', name=u'persons', rank=u'4', value=u'Putin, Vladimir V')]
Row(is_major=u'N', name=u'subject', rank=u'1', value=u'Cyberwarfare and Defense')
5206
4


In [52]:
rddPersons = sc.textFile(cwd+"/PersonsCollection.txt")
rddSubjects = sc.textFile(cwd+"SubjectCollection.txt")
rddOrganizations = sc.textFile(cwd+"OrganizationCollection.txt")

print cwd

/Users/Enzo/anaconda2/CSC_599_WORK


In [53]:
rddPersons.take(10)

[u'trump  donald j',
 u'strine  leo e jr',
 u'syndergaard  noah',
 u'montero  rafael',
 u'collins  terry l',
 u'soros  george',
 u'orban  viktor',
 u'de blasio  bill',
 u'campbell  thomas p',
 u'coburn  erin']

In [33]:
wc = rddPersons.map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y: x+y)

wc.take(10)

[(u'reider  jonah', 1),
 (u'shannon  christopher', 1),
 (u'litiatco  armando', 2),
 (u'zeid ra ad zeid al hussein', 1),
 (u'magoon  daniel j', 1),
 (u'chozick  amy', 1),
 (u'wells  jane', 2),
 (u'afineevsky  evgeny         ', 2),
 (u'browder  ben', 1),
 (u'garrett  myles         ', 3)]

In [45]:
import heapq

def mapper2(counts):
    yield heapq.nlargest(100, counts, key=lambda x: x[1])
    
def reducer2(top31,top32):
    return heapq.nlargest(100, top31 + top32, key=lambda x: x[1])


top3 = wc.mapPartitions(mapper2).reduce(reducer2)
type(top3)

list

In [46]:
top3

[(u'trump  donald j', 3252),
 (u'obama  barack', 292),
 (u'gorsuch  neil m', 133),
 (u'putin  vladimir v', 133),
 (u'sessions  jefferson b iii', 120),
 (u'de blasio  bill', 117),
 (u'tillerson  rex w', 89),
 (u'cuomo  andrew m', 88),
 (u'assad  bashar al ', 88),
 (u'ryan  paul d jr', 84),
 (u'bannon  stephen k', 81),
 (u'xi jinping', 76),
 (u'trump  ivanka', 73),
 (u'devos  elizabeth         ', 72),
 (u'erdogan  recep tayyip', 70),
 (u'le pen  marine', 69),
 (u'kim jong un', 68),
 (u'spicer  sean m         ', 67),
 (u'flynn  michael t', 64),
 (u'may  theresa m', 63),
 (u'clinton  hillary rodham', 63),
 (u'o reilly  bill', 61),
 (u'conway  kellyanne', 57),
 (u'kushner  jared', 54),
 (u'merkel  angela', 51),
 (u'netanyahu  benjamin', 48),
 (u'pence  mike', 46),
 (u'anthony  carmelo', 44),
 (u'colbert  stephen', 43),
 (u'pruitt  scott', 40),
 (u'trump  melania', 40),
 (u'comey  james b', 39),
 (u'duterte  rodrigo', 39),
 (u'mcconnell  mitch', 38),
 (u'mattis  james n', 38),
 (u'macron  em

#### Look for   =>   Top100People.txt